In [7]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
df_transaction = pd.read_csv('../data/processed_data/transactions.csv')
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40171 entries, 0 to 40170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                40171 non-null  object 
 1   credit              40171 non-null  object 
 2   transaction_detail  40171 non-null  object 
 3   transaction_id      40169 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.2+ MB


In [9]:
df_transaction['credit'] = df_transaction['credit'].str.replace('.', '').astype(float)
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40171 entries, 0 to 40170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                40171 non-null  object 
 1   credit              40171 non-null  float64
 2   transaction_detail  40171 non-null  object 
 3   transaction_id      40169 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.2+ MB


In [15]:
# Define bins and labels
bins = [0, 10000, 20000, 50000, 100000, 500000, 1000000, 5000000, 10000000, 50000000, 100000000, 500000000, float('inf')]
labels = ['[0,10k]', '[10k,20k]', '[20k,50k]', '[50k,100k]', '[100k,500k]', '[500k,1M]', 
          '[1M,5M]', '[5M,10M]', '[10M,50M]', '[50M,100M]', '[100M,500M]', '[>500M]']

# Create a new column 'bin' with the bin labels
df_transaction['bin'] = pd.cut(df_transaction['credit'], bins=bins, labels=labels, right=False)

# Group by the 'bin' column and count the number of transactions in each bin
bin_counts = df_transaction['bin'].value_counts().sort_index()

# Create a new DataFrame with the bin labels and counts
df_bin_counts = pd.DataFrame({'Khoảng mệnh giá': bin_counts.index, 'Số lượng giao dịch': bin_counts.values})

print(df_bin_counts)


   Khoảng mệnh giá  Số lượng giao dịch
0          [0,10k]                 436
1        [10k,20k]                 605
2        [20k,50k]                 936
3       [50k,100k]                2965
4      [100k,500k]               20714
5        [500k,1M]                6994
6          [1M,5M]                6225
7         [5M,10M]                 666
8        [10M,50M]                 517
9       [50M,100M]                  71
10     [100M,500M]                  42
11         [>500M]                   0
